<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C8-Common_Trojaning_Tasks_On_Windows/Keylogging_For_Fun_And_Keystrokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Common Trojaning Tasks On Windows__*

When you deploy a trojan, you may want to perform a few common tasks with it: Grab keystrokes, take screenshots, and execute shellcode to provide an interactive session to tools like __CANVAS__ or __Metasploit__. This chapter focuses on performing these tasks on Windows systems. We'll wrap things up with some sandbox detection techniques to determine if we are running within an antivirus or forensics sandbox. These modules will be easy to modify and will work within the trojan framework developed in Chapter 7. In later chapters, we'll explore privilege escalation techniques that you can deploy with your trojan. Each technique comes with its own challenges and probability of being caught, either by the end user or an antivirus solution.
We recommend that you carefully model your target after you've implanted your trojan so that you can test the modules in your lab before trying them on a live target. Let's get started by creating a simple keylogger.

### *__Keylogging for Fun and Keystrokes__*

__Keylogging__, the use of a concealed program to record consecutive keystrokes, is one of the oldest tricks in the book, and it's still employed with various levels of stealth today. Attackers still use it because it's extremely effective at capturing sensitive information such as credentials or conversations.

An excellent Python library named __PyWinHook__ enables us to easily trap all keyboard events (https://pypi.org/project/pyWinhook/). __PyWinHook__ is a fork of the orginal __PyHook__ library and is updated to support Python 3. It takes advantage of the native Windows function __SetWindowsHookEx__, which allows us to install a user-defined function to be called for certain Windows events. By registering a hook